In [30]:
import torch
import torch.nn as nn
import numpy as np
import os
import sys
import cv2
from PIL import Image
import easydict
sys.path.append('../Whatiswrong')
import Extract
import utils
import torch.nn.functional as F

In [5]:
import torchvision.models as models

In [6]:
models.resnet50()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
class Basemodel(nn.Module):
    def __init__(self, opt):
        super(Basemodel, self).__init__()
        self.encoder = Resnet_EFIFSTR(with_lstm=True)
        self.decoder = 
        
    def forward(self, img)

In [46]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class AsterBlock(nn.Module):

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(AsterBlock, self).__init__()
        self.conv1 = conv1x1(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class Resnet_EFIFSTR(nn.Module):
    def __init__(self, with_lstm=False, n_group=1):
        super(Resnet_EFIFSTR, self).__init__()
        self.with_lstm = with_lstm
        self.n_group= n_group
        
        in_channels=3
        self.layer0 = nn.Sequential(nn.Conv2d(in_channels, 32, kernel_size=(3,3), stride=1, padding=1, bias=False),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(inplace=True))
        self.inplanes = 32
        self.layer1 = self._make_layer(32, 3, [2,2])
        self.layer2 = self._make_layer(64, 4, [2,2])
        self.layer3 = self._make_layer(128, 6, [2,1])
        self.layer4 = self._make_layer(256, 6, [1,1])
        self.layer5 = self._make_layer(512, 3, [1,1])
        
        if with_lstm:
            self.rnn = nn.LSTM(512, 256, bidirectional=True, num_layers=2, batch_first=True)
            self.out_planes = 2 * 256
        else:
            self.out_planes = 512
            
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
    def _make_layer(self, planes, blocks, stride):
        downsample = None
        if stride !=[1,1] or self.inplanes != planes:
            downsample = nn.Sequential(conv1x1(self.inplanes, planes, stride),
                                      nn.BatchNorm2d(planes))
            
        layers = []
        layers.append(AsterBlock(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(AsterBlock(self.inplanes, planes))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x0 = self.layer0(x)
        x1 = self.layer1(x0)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)
        x5 = self.layer5(x4)
        
        batch_size, channels, feature_h, feature_w = x5.shape
        cnn_feat = F.max_pool2d(x5, (feature_h, 1))
        cnn_feat = cnn_feat.permute(0, 3, 1, 2).squeeze(3)
        
        if self.with_lstm:
            rnn_feat, _ = self.rnn(cnn_feat)
            return rnn_feat
        else:
            return cnn_feat
                   

In [47]:
extract = Resnet_EFIFSTR(with_lstm=True)

In [48]:
inputt = torch.FloatTensor(1,3, 48, 160)

In [49]:
ext_res = extract(inputt )

In [50]:
ext_res.shape

torch.Size([1, 40, 512])